<a href="https://colab.research.google.com/github/mikenew01/100-days-of-code/blob/master/C%C3%B3pia_de_MODEL_TRAINING___chakras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**TRAIN NEW MODEL**

In [ ]:
#@title [1]Verify GPU is connected
!nvidia-smi

In [ ]:
#@title [2] Installs dependencies (You can ignore errors here..)
!python -m pip install -U pip wheel
%pip install -U ipython
%pip install -U git+https://github.com/34j/so-vits-svc-fork@9c8df717ae433fe3f3e2ef6ee49a1a165c66cf03
%pip install mega.py

%cd /content
!rm -r sample_data
!mkdir dataset_raw

In [ ]:
#@title [3] Download dataset from google drive
#@markdown **Enter Google Drive link to dataset"
import os
import gdown

!pip install gdown

%cd /content
os.chdir("/content")

name = "trainingdata"
google_drive_link = "drive_link_to_dataset"  #@param {type: "string"}
destination_filename = name + ".zip"

# Extract the file ID from the Google Drive link
file_id = google_drive_link.split("id=")[-1] if "id=" in google_drive_link else google_drive_link.split("/")[-2]
# Construct the direct download link
url = f"https://drive.google.com/uc?id={file_id}"
# Download the file using gdown
gdown.download(url, destination_filename, quiet=False)

files_before = os.listdir()
!unzip "trainingdata.zip"
files_after = os.listdir()
new_file = list(set(files_after) - set(files_before))[0]

!rm {destination_filename}

!mv {new_file} dataset_raw

In [ ]:
#@title [4] Automatic preprocessing

!svc pre-resample
!svc pre-config
!svc pre-hubert
!rm -r /content/dataset_raw

In [ ]:
#@title [OPTIONAL] Run this to import previous model to continue training
#@markdown **Enter Google Drive link to previously saved model**
import os
import gdown

!pip install gdown

%cd /content
os.chdir("/content")

name = "model"
google_drive_link = "drive_link_to_saved_model"  #@param {type: "string"}
destination_filename = name + ".zip"

# Extract the file ID from the Google Drive link
file_id = google_drive_link.split("id=")[-1] if "id=" in google_drive_link else google_drive_link.split("/")[-2]
# Construct the direct download link
url = f"https://drive.google.com/uc?id={file_id}"
# Download the file using gdown
gdown.download(url, destination_filename, quiet=False)

files_before = os.listdir()
!unzip "model.zip"
files_after = os.listdir()
new_file = list(set(files_after) - set(files_before))[0]

!rm model.zip
!mkdir -p logs/44k
!cp -f {new_file}/config.json configs/44k
!mv -f {new_file}/config.json logs/44k
!mv {new_file}/*.pth logs/44k/
!rm -r {new_file}


In [ ]:
#@title [5] Train Model
!svc train

In [ ]:
#@title [6] Save your model to Google Drive

import os
import shutil

from google.colab import drive
drive.mount("/content/drive/")


model_name = "save_file_name" #@param {type: "string"}
!mkdir {model_name}

# Set your source and destination directories
source_dir = "/content/logs/44k/"
destination_dir = "/content"

# List all files in the source directory
files = os.listdir(source_dir)

# Filter files starting with 'G_' and 'D_'
g_files = [f for f in files if f.startswith("G_")]
d_files = [f for f in files if f.startswith("D_")]

# Find the highest numbered G_ and D_ files
g_max = max(g_files, key=lambda x: int(x.split("_")[-1].split(".")[0]))
d_max = max(d_files, key=lambda x: int(x.split("_")[-1].split(".")[0]))

# Copy the files to the destination directory
shutil.copy(os.path.join(source_dir, g_max), os.path.join(destination_dir, g_max))
shutil.copy(os.path.join(source_dir, d_max), os.path.join(destination_dir, d_max))

!mv *.pth {model_name}
!cp logs/44k/config.json {model_name}

!zip -r "{model_name}.zip" {model_name}

!cp "{model_name}.zip" drive/MyDrive